In [1]:
import os
import bs4
import requests
import json

In [2]:
links = [
  "https://www.momjunction.com/articles/family-feud-questions_00474806/",
  "https://hobbylark.com/party-games/list-of-family-feud-questions",
  "https://hobbylark.com/party-games/pop-culture-family-feud-questions",
  "https://hobbylark.com/party-games/family-feud-adult-questions"
]

In [3]:
def bs_len(bs):
  return len(list(bs))

def nested_access(arr, *args):
  if len(args) > 1:
    return nested_access(list(arr)[args[0]], *args[1:])
  return list(arr)[args[0]]

In [4]:
select = 1
link = links[select]

**Web 1**

In [4]:
page = requests.get(link).content
soup = bs4.BeautifulSoup(page, 'html.parser')
html = list(list(soup.children)[1].children)[1]
content = nested_access(html.children, 4, 0, 0, 0, 1, 0)

questions = []
answers = []

prev = {
  "name": None,
  "text": None
}
for i, tag in enumerate(content):
  if (i < 44 or i > 354): continue
  if (tag.name == "p"):
    if prev["name"] == tag.name:
      print("Alert: Recurring <p> tag. Overwriting previous question")
      questions[-1] = tag.text
    else:
      questions += [tag.text]
      prev["name"] = tag.name
      prev["text"] = tag.text
  if (tag.name == "ul"):
    answers += [[option.text for option in tag]]
    prev["name"] = tag.name
    prev["text"] = tag.text
assert len(questions) == len(answers)

q1 = [{"question": q.split(" ", 1)[1], "answer": a} for (q, a) in zip(questions, answers)]
json.dump(q1, open("output/0.json", "w"), indent=2)

**Web 2**

In [38]:
select = 3
link = links[select]
page = requests.get(link).content
soup = bs4.BeautifulSoup(page, 'html.parser')

all_mod_table = soup.find_all(class_="moduleTable")

questions = []
answers = []
for module in all_mod_table:
  try:
    question = module.find_all("h2")[0].text
    answer_grp = module.findAll(class_="firstCell")
    q_answers = [i.text.strip() for i in answer_grp if i.text.strip() != '']
    
    questions += [question]
    answers += [q_answers]
  except IndexError as e:
    continue
assert len(questions) == len(answers)

q2 = [{"question": q, "answer": a} for (q, a) in zip(questions, answers)]
json.dump(q2, open("output/" + str(select) +".json", "w"), indent=2)